In [1]:
import pandas as pd
import datetime
from datetime import date
import matplotlib.pyplot as plt

In [2]:
# Clean list of county
ca_county_path = 'Resources/ca_county_list.csv'
ca_county = pd.read_csv(ca_county_path)
ca_county = ca_county.rename(columns={'NAME': 'Counties'})

# Store clean data in csv file
ca_county.to_csv('Resources/ca_county_list_clean.csv', index=False, header=True)

In [3]:
# Study data files
fire_data_path = 'Resources/California_Fire_Incidents_original.csv'
ca_county_path = 'Resources/ca_county_list_clean.csv'

# Read in csv file
fire_df = pd.read_csv(fire_data_path)
ca_county_df = pd.read_csv(ca_county_path)

In [4]:
# Combine the data into a single dataset
merged_fire_df = pd.merge(ca_county, fire_df, how='left', left_on='Counties', right_on='Counties')

# Converting dtypes
merged_fire_df['ArchiveYear'] = merged_fire_df['ArchiveYear'].map('{:.0f}'.format)

merged_fire_df.head()

,Counties,AcresBurned,Active,AdminUnit,AirTankers,ArchiveYear,CalFireIncident,CanonicalUrl,ConditionStatement,ControlStatement,...,SearchKeywords,Started,Status,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,Updated,WaterTenders
0,Alameda,240.0,False,CAL FIRE Santa Clara Unit,NaN,2013,True,/incidents/2013/6/8/vasco-fire/,NaN,NaN,...,Vasco Fire; June 2013; Vasco Road; North Vasco...,2013-06-08T10:16:00Z,Finalized,NaN,NaN,NaN,NaN,2ad509e4-99f1-4e1a-81eb-839f772f9c45,2013-06-08T13:25:00Z,NaN
1,Alameda,150.0,False,CAL FIRE Santa Clara Unit,NaN,2013,True,/incidents/2013/10/4/highland-fire/,NaN,NaN,...,Highland FIre; October 2013; Alameda County; H...,2013-10-04T07:35:00Z,Finalized,NaN,NaN,NaN,NaN,6a48000a-6198-46dd-9a91-0b0159183145,2013-10-04T10:30:00Z,NaN
2,Alameda,50.0,False,CAL FIRE Santa Clara Unit,NaN,2013,True,/incidents/2013/7/4/grant-fire/,NaN,NaN,...,Grant Fire; July 2013; Grant Line Road; I-580;...,2013-07-04T10:00:00Z,Finalized,NaN,NaN,NaN,NaN,07fb8b11-f39b-4cd0-8a27-2d4fef7fe680,2013-07-04T11:30:00Z,NaN
3,Alameda,38.0,False,CAL FIRE Santa Clara Unit / Alameda County Fire,NaN,2013,True,/incidents/2013/7/6/fallon-fire/,NaN,NaN,...,Fallon Fire; July 2013; Fallon Road; Camino Ta...,2013-07-06T13:09:00Z,Finalized,NaN,NaN,NaN,NaN,4c04a4e5-f580-4e62-bb53-2575c72d29dc,2013-07-06T14:30:00Z,NaN
4,Alameda,2700.0,False,CAL FIRE Santa Clara Unit,NaN,2015,True,/incidents/2015/8/19/tesla-fire/,Unit is now in control of incident.,NaN,...,Tesla Fire; Tesla and Correl Hollow; Livermore...,2015-08-19T14:45:00Z,Finalized,NaN,NaN,NaN,NaN,8fc920eb-ca45-4d77-9a92-1326eca602d6,2015-08-22T18:30:00Z,1.0


In [5]:
# Clean data
drop_column_df = merged_fire_df.drop(columns=[
    "Active", 
    "CanonicalUrl", 
    "ConditionStatement", 
    "FuelType", 
    "Location", 
    "ControlStatement",
    "CalFireIncident", 
    "AdminUnit", 
    "PercentContained", 
    "Status", 
    "SearchDescription", 
    "SearchKeywords", 
    "Public", 
    "Updated"], inplace=False)

drop_column_df.head()

# Store clean data in csv file
drop_column_df.to_csv('Resources/california_fire_merged.csv', index=False, header=True)

In [6]:
original_ca_fire_df = drop_column_df.copy()

# Get start and end date and month data in each row
start = list(original_ca_fire_df['Started'])
end = list(original_ca_fire_df['Extinguished'])
start_date = []
end_date = []
new_format = "%Y-%m-%d"

for sd in start:
    try:
        s_date = datetime.datetime.strptime(sd[0:10], new_format).strftime(new_format)
        start_date.append(s_date)
    except:
        start_date.append(None)
        
for ed in end:
    try:
        e_date = datetime.datetime.strptime(ed[0:10], new_format).strftime(new_format)
        end_date.append(e_date)
    except:
        end_date.append(None)

# No. of day that fire incident happens
days = []

for d in range(len(end_date)):
    try:
        e_date = end_date[d]
        date_e = date(int(e_date[0:4]), int(e_date[5:7]), int(e_date[8:10]))
        s_date = start_date[d]
        date_s = date(int(s_date[0:4]), int(s_date[5:7]), int(s_date[8:10]))
        day = date_e - date_s
        days.append(day.days)
    except:
        days.append(None)

# Started month
months = []

for m in range(len(start_date)):
    try:
        s_month = start_date[m]
        month = int(s_month[5:7])
        months.append(str(month))
    except:
        months.append(None)
        
# Add to dataframe
original_ca_fire_df['Started Date'] = start_date
original_ca_fire_df['Ended Date'] = end_date
original_ca_fire_df['Started Month'] = months
original_ca_fire_df['Days Burned'] = days

original_ca_fire_df.head()

,Counties,AcresBurned,AirTankers,ArchiveYear,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,...,StructuresDamaged,StructuresDestroyed,StructuresEvacuated,StructuresThreatened,UniqueId,WaterTenders,Started Date,Ended Date,Started Month,Days Burned
0,Alameda,240.0,NaN,2013,1,NaN,NaN,NaN,2013-06-08T13:25:00Z,NaN,...,NaN,NaN,NaN,NaN,2ad509e4-99f1-4e1a-81eb-839f772f9c45,NaN,2013-06-08,2013-06-08,6,0.0
1,Alameda,150.0,NaN,2013,1,NaN,NaN,NaN,2013-10-04T10:30:00Z,NaN,...,NaN,NaN,NaN,NaN,6a48000a-6198-46dd-9a91-0b0159183145,NaN,2013-10-04,2013-10-04,10,0.0
2,Alameda,50.0,NaN,2013,1,NaN,NaN,NaN,2013-07-04T11:30:00Z,NaN,...,NaN,NaN,NaN,NaN,07fb8b11-f39b-4cd0-8a27-2d4fef7fe680,NaN,2013-07-04,2013-07-04,7,0.0
3,Alameda,38.0,NaN,2013,1,NaN,NaN,NaN,2013-07-06T14:30:00Z,NaN,...,NaN,NaN,NaN,NaN,4c04a4e5-f580-4e62-bb53-2575c72d29dc,NaN,2013-07-06,2013-07-06,7,0.0
4,Alameda,2700.0,NaN,2015,1,NaN,NaN,5.0,2015-08-22T18:30:00Z,NaN,...,NaN,NaN,NaN,NaN,8fc920eb-ca45-4d77-9a92-1326eca602d6,1.0,2015-08-19,2015-08-22,8,3.0


In [7]:
# set up additional columns to hold information
original_ca_fire_df['AvgAcresBurned'] = original_ca_fire_df['AcresBurned']
original_ca_fire_df['AvgAirTankers'] = original_ca_fire_df['AirTankers']
original_ca_fire_df['AvgCrewsInvolved'] = original_ca_fire_df['CrewsInvolved']
original_ca_fire_df['AvgDozers'] = original_ca_fire_df['Dozers']
original_ca_fire_df['AvgEngines'] = original_ca_fire_df['Engines']
original_ca_fire_df['AvgFatalities'] = original_ca_fire_df['Fatalities']
original_ca_fire_df['AvgHelicopters'] = original_ca_fire_df['Helicopters']
original_ca_fire_df['AvgInjuries'] = original_ca_fire_df['Injuries']
original_ca_fire_df['AvgPersonnelInvolved'] = original_ca_fire_df['PersonnelInvolved']
original_ca_fire_df['AvgStructuresDamaged'] = original_ca_fire_df['StructuresDamaged']
original_ca_fire_df['AvgStructuresDestroyed'] = original_ca_fire_df['StructuresDestroyed']
original_ca_fire_df['AvgStructuresEvacuated'] = original_ca_fire_df['StructuresEvacuated']
original_ca_fire_df['AvgStructuresThreatened'] = original_ca_fire_df['StructuresThreatened']
original_ca_fire_df['AvgWaterTenders'] = original_ca_fire_df['WaterTenders']
original_ca_fire_df.head()

,Counties,AcresBurned,AirTankers,ArchiveYear,CountyIds,CrewsInvolved,Dozers,Engines,Extinguished,Fatalities,...,AvgEngines,AvgFatalities,AvgHelicopters,AvgInjuries,AvgPersonnelInvolved,AvgStructuresDamaged,AvgStructuresDestroyed,AvgStructuresEvacuated,AvgStructuresThreatened,AvgWaterTenders
0,Alameda,240.0,NaN,2013,1,NaN,NaN,NaN,2013-06-08T13:25:00Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Alameda,150.0,NaN,2013,1,NaN,NaN,NaN,2013-10-04T10:30:00Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Alameda,50.0,NaN,2013,1,NaN,NaN,NaN,2013-07-04T11:30:00Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Alameda,38.0,NaN,2013,1,NaN,NaN,NaN,2013-07-06T14:30:00Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alameda,2700.0,NaN,2015,1,NaN,NaN,5.0,2015-08-22T18:30:00Z,NaN,...,5.0,NaN,NaN,NaN,18.0,NaN,NaN,NaN,NaN,1.0


In [ ]:
id_count = original_ca_fire_df['UniqueId'].value_counts().to_frame().reset_index()
uid_count = id_count.loc[id_count['UniqueId'] > 1]
df_index = []
len_list = []

for i in uid_count['index']:
    u_df = original_ca_fire_df.loc[original_ca_fire_df['UniqueId']==i]
    index_list = u_df.index
    df_index.append(index_list)
    
for n in range(len(df_index)):
    df_index_id = df_index[n]
    for x in df_index_id:
        original_ca_fire_df.loc[x, ['AvgAcresBurned',
                                    'AvgAirTankers',
                                    'AvgCrewsInvolved',
                                    'AvgDozers',
                                    'AvgEngines',
                                    'AvgFatalities',
                                    'AvgHelicopters',
                                    'AvgInjuries',
                                    'AvgPersonnelInvolved',
                                    'AvgStructuresDamaged',
                                    'AvgStructuresDestroyed',
                                    'AvgStructuresEvacuated',
                                    'AvgStructuresThreatened',
                                    'AvgWaterTenders']] = original_ca_fire_df.loc[x, ['AvgAcresBurned',
                                                                                      'AvgAirTankers',
                                                                                      'AvgCrewsInvolved',
                                                                                      'AvgDozers',
                                                                                      'AvgEngines',
                                                                                      'AvgFatalities',
                                                                                      'AvgHelicopters',
                                                                                      'AvgInjuries',
                                                                                      'AvgPersonnelInvolved',
                                                                                      'AvgStructuresDamaged',
                                                                                      'AvgStructuresDestroyed',
                                                                                      'AvgStructuresEvacuated',
                                                                                      'AvgStructuresThreatened',
                                                                                      'AvgWaterTenders']].div(len(df_index_id))
original_ca_fire_df.head()

In [9]:
# Store clean data in csv file
original_ca_fire_df.to_csv('output_data/california_fire_cleaned.csv', index=False, header=True)